# Instrument object
This section shows the majority of the features implemented for the instrument object in McStasScript.

## Initialization
An instrument object is created with the [McStas_instr](../_autosummary/mcstasscript.interface.instr.McStas_instr.rst) or [McXtrace_instr](../_autosummary/mcstasscript.interface.instr.McXtrace_instr.rst) class in the instr module. When an instrument object is created the only required argument is the name of the instrument which will be used for the instrument filename. There are however a number of keyword arguments that can be used to provide more information and alter the behavior.

| Keyword argument | Type | Default | Description |
| --- | --- | --- | --- |
| author | str |"Python Instrument Generator" | Name that will appear as author in instrument files |
| origin | str |"ESS DMSC" | String that will appear as origin in instrument files |
| input_path | str | "." | Folder which is considered workspace for McStas / McXtrace |
| output_path | str | instrument_name | Name of data folder written by simulation |
| package_path | str | | Can be set to manually specify location of McStas/McXtrace installation |
| executable_path | str |  | Can be set to manually specify location of mcrun/mxrun executable |
| ncount | int, float | 1E6 | Sets the ncount used for simulations |
| mpi | int |  | Sets the number of MPI threads used for simulations |
| force_compile | bool | True | Whether to force compilation before each run or not |
| parameters | ParameterContainer |  | Set of parameters for initialized instrument |

In [1]:
from mcstasscript.interface import instr, functions, plotter

In [2]:
instrument = instr.McStas_instr("instr_name", author="Mads Bertelsen", origin="DMSC")
instrument_w_settings = instr.McStas_instr("instr_name", ncount=3E6, output_path="new_folder")

### Using settings method
The instrument object has a *setting* method which can update some settings after initialization. The current settings can always be viewed with *show_settings*.

| Keyword argument | Type | Default | Description |
| --- | --- | --- | --- |
| output_path | str | instrument_name | Name of data folder written by simulation |
| package_path | str | | Can be set to manually specify location of McStas/McXtrace installation |
| executable_path | str |  | Can be set to manually specify location of mcrun/mxrun executable |
| ncount | int, float | 1E6 | Sets the ncount used for simulations |
| mpi | int |  | Sets the number of MPI threads used for simulations |
| seed |  |  | Sets the seed of the simulation |
| force_compile | bool | True | Whether to force compilation before each run or not |
| custom_flags | str |  | String with custom flags for mcrun/mxrun command |

In [3]:
instrument.show_settings()
instrument_w_settings.show_settings()

Instrument settings:
  output_path:      instr_name_data
  run_path:         .
  package_path:     /Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1
  executable_path:  /Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1/bin/
  executable:       mcrun
  force_compile:    True
Instrument settings:
  ncount:           3.00e+06
  output_path:      new_folder
  run_path:         .
  package_path:     /Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1
  executable_path:  /Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1/bin/
  executable:       mcrun
  force_compile:    True


In [4]:
instrument.settings(mpi=4, seed=300)
instrument.show_settings()

Instrument settings:
  mpi:              4
  seed:             300
  output_path:      instr_name_data
  run_path:         .
  package_path:     /Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1
  executable_path:  /Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1/bin/
  executable:       mcrun
  force_compile:    True


## Parameters
Instrument parameters can be added with *add_parameters* which returns a parameter object.

In [5]:
wavelength = instrument.add_parameter("wavelength", comment="Wavelength in AA")
print(wavelength)
wavelength.value = 5
print(wavelength)

Parameter named: 'wavelength' without set value.
 Wavelength in AA

Parameter named: 'wavelength' with value: 5
 Wavelength in AA



## Initialize section
One of the great advantages for the McStas / McXtrace packages is the initialize section of the instrument where calculations can be performed before the ray-tracing simulation starts. One could for example calculate appropriate angles to reach a certain Bragg peak at a given wavelength. This would involve defining some declare variables, using these in the initialize section and then assigning them as component inputs.

In McStasScript many calculations can be performed directly in Python, and so typically the initialize section is used less, but it is still useful and available through McStasScript.

The instrument object has the method *append_initialize* which adds a line of code to the initialize. This line is copied directly into the instrument file, so it follows C syntax. Remember the semicolon! In addition there is *add_declare_var* to specify the declared variables needed. When declare variables are defined an object is returned which can be used when referring to that variable.

In [6]:
wavenumber = instrument.add_declare_var("double", "wavenumber")
instrument.append_initialize("wavenumber = 2*PI/wavelength;")

In [7]:
print(instrument.initialize_section)

// Start of initialize for generated instr_name
wavenumber = 2*PI/wavelength;



## Finally section
The finally section works exactly as the initialize section, but is executed after the ray-tracing simulation. Add a line to it with *append_finally*.

In [8]:
instrument.append_finally('printf(\"Thanks for using McStasScript!\\n\");')
print(instrument.finally_section)

// Start of finally for generated instr_name
printf("Thanks for using McStasScript!\n");



## Help features
There are a few methods built into the instrument class that helps the user, these are:

- *show_components*
- *component_help*

### show_components
The *show_components* method shows the component categories, and if called with the name of a category, will show all components in the specified category. The categories can include the work directory if any components are located there.

In [9]:
instrument.show_components()

Here are the available component categories:
 contrib
 misc
 monitors
 obsolete
 optics
 samples
 sources
 union
Call show_components(category_name) to display


In [10]:
instrument.show_components("optics")

Here are all components in the optics category.
 Absorber                 Guide_gravity          Pol_bender
 Arm                      Guide_simple           Pol_constBfield
 Beamstop                 Guide_tapering         Pol_guide_mirror
 Bender                   Guide_wavy             Pol_guide_vmirror
 Collimator_linear        He3_cell               Pol_mirror
 Collimator_radial        Mask                   Refractor
 Derotator                Mirror                 Rotator
 Diaphragm                Monochromator_curved   Selector
 DiskChopper              Monochromator_flat     Set_pol
 Elliptic_guide_gravity   Monochromator_pol      Slit
 FermiChopper             PolAnalyser_ideal      V_selector
 Filter_gen               Pol_Bfield             Virtual_mcnp_ss_Guide
 Guide                    Pol_Bfield_stop        Vitess_ChopperFermi
 Guide_anyshape           Pol_FieldBox           
 Guide_channeled          Pol_SF_ideal           


### component_help
The *component_help* method can show the parameters of any component the instrument object knows about, although not necessarily used in the instrument.

In [11]:
instrument.component_help("Guide")

 ___ Help Guide _____________________________________________________________________
|optional parameter|required parameter|default value|user specified value|
reflect = 0 [str] // Reflectivity file name. Format <q(Angs-1) R(0-1)>
w1 [m] // Width at the guide entry
h1 [m] // Height at the guide entry
w2 = 0.0 [m] // Width at the guide exit
h2 = 0.0 [m] // Height at the guide exit
l [m] // length of guide
R0 = 0.99 [1] // Low-angle reflectivity
Qc = 0.0219 [AA-1] // Critical scattering vector
alpha = 6.07 [AA] // Slope of reflectivity
m = 2.0 [1] // m-value of material. Zero means completely absorbing. glass/SiO2 
               Si Ni Ni58 supermirror Be Diamond m=  0.65 0.47 1 1.18 2-6 1.01 1.12 
W = 0.003 [AA-1] // Width of supermirror cut-off
-------------------------------------------------------------------------------------


## Adding components
One adds components to the instrument using *add_component* which takes the name of the component instance for the instrument, followed by the name of the component in the library. When adding a component, a [component](../_autosummary/mcstasscript.helper.mcstas_objects.Component.rst) object is returned, and how these can be manipulated is discussed on the [component object page](component_object.ipynb). Notice that it is not allowed to add two components with the same instance name, meaning rerunning this cell would raise an exception. 

In [12]:
source = instrument.add_component("source", "Source_div")
source.set_parameters(xwidth=0.1, yheight=0.1, focus_aw=3.0, focus_ah=2.0, 
                      lambda0=wavelength, dlambda="0.1*wavelength")

In [13]:
print(source)

COMPONENT source = Source_div
  xwidth = 0.1 [m]
  yheight = 0.1 [m]
  focus_aw = 3.0 [deg]
  focus_ah = 2.0 [deg]
  lambda0 = wavelength [Ang]
  dlambda = 0.1*wavelength [Ang]
AT [0, 0, 0] ABSOLUTE


In [14]:
instrument.print_components()

source Source_div AT (0, 0, 0) ABSOLUTE


There are a number of keyword arguments allowed when adding a component. These will mainly be discussed on the [component object page](component_object.ipynb), but a few are relevant for the instrument, because the handle in what order components are sequenced in the instrument. To illustrate this we add a slit and a guide to the instrument at reasonable positions. Notice these new components are added at the end of the instrument.

In [15]:
slit = instrument.add_component("source_slit", "Slit", AT=2, RELATIVE=source)
slit.set_parameters(xwidth=0.015, yheight=0.015)

guide = instrument.add_component("guide", "Guide", AT=0.1, RELATIVE=slit)
guide.set_parameters(w1=0.03, h1=0.03, l=10.0)

In [16]:
instrument.print_components()

source      Source_div AT      (0, 0, 0)   ABSOLUTE            
source_slit Slit       AT      (0, 0, 2)   RELATIVE source     
guide       Guide      AT      (0, 0, 0.1) RELATIVE source_slit


The order of components is important in a McStas/McXtrace simulation as each will affect the ray state in the sequence shown with *print_components*. If one wants to add a component between the source and the slit, this can be done with the *before* or *after* keyword.

In [17]:
monitor = instrument.add_component("PSD", "PSD_monitor", after="source")
monitor.set_AT(1.9, RELATIVE=source)
monitor.set_parameters(xwidth=0.1, yheight=0.1, filename='"PSD.dat"')

instrument.print_components()

source      Source_div  AT      (0, 0, 0)   ABSOLUTE            
PSD         PSD_monitor AT      (0, 0, 1.9) RELATIVE source     
source_slit Slit        AT      (0, 0, 2)   RELATIVE source     
guide       Guide       AT      (0, 0, 0.1) RELATIVE source_slit


The PSD monitor was inserted after the source, this could also be accomplished with the before keyword argument.
```
before="source_slit"
```
It is important to note that the McStas instrument file is read sequentially, so the position of the PSD monitor can not be relative to a later component, but must only refer to earlier components. At this point in development it is not possible to reorder components in the instrument object.

## Making a component copy
It is possible to copy an existing component using the *copy_component* method. This can reduce both the amount of typing necessary, but also the risk of making a mistake. Here the guide is copied and placed a bit after the end of the first guide, with a small rotation.

In [18]:
guide2 = instrument.copy_component("guide_2", "guide")
guide2.set_AT(guide.l + 0.01, RELATIVE=guide)
guide2.set_ROTATED([0, 0.5, 0], RELATIVE=guide)
print(guide2)

COMPONENT guide_2 = Guide
  w1 = 0.03 [m]
  h1 = 0.03 [m]
  l = 10.0 [m]
AT [0, 0, 10.01] RELATIVE guide
ROTATED [0, 0.5, 0] RELATIVE guide


## Getting components

It is always possible to retrieve the component objects corresponding to components in the instrument with the *get_component* and *get_last_component* methods.

In [19]:
my_source = instrument.get_component("source")
print(my_source)

COMPONENT source = Source_div
  xwidth = 0.1 [m]
  yheight = 0.1 [m]
  focus_aw = 3.0 [deg]
  focus_ah = 2.0 [deg]
  lambda0 = wavelength [Ang]
  dlambda = 0.1*wavelength [Ang]
AT [0, 0, 0] ABSOLUTE


In [20]:
last_component = instrument.get_last_component()
print(last_component)

COMPONENT guide_2 = Guide
  w1 = 0.03 [m]
  h1 = 0.03 [m]
  l = 10.0 [m]
AT [0, 0, 10.01] RELATIVE guide
ROTATED [0, 0.5, 0] RELATIVE guide


## Run the simulation
The simulation is executed with a call to the *backengine* method, which will return the generated data. If the simulation fails, the method returns None.

In [21]:
data = instrument.backengine()

INFO: Using directory: "/Users/madsbertelsen/PaNOSC/McStasScript/github/McStasScript/docs/source/user_guide/instr_name_data_23"
INFO: Regenerating c-file: instr_name.c
CFLAGS=
INFO: Recompiling: ./instr_name.out
mccode-r.c:1880:1: warning: non-void function does not return a value in all control paths [-Wreturn-type]
} /* mcsiminfo_init */
^
mccode-r.c:2837:3: warning: expression result unused [-Wunused-value]
  *t0;
  ^~~
2 warnings generated.
INFO: ===
INFO: Placing instr file copy instr_name.instr in dataset /Users/madsbertelsen/PaNOSC/McStasScript/github/McStasScript/docs/source/user_guide/instr_name_data_23

Simulation 'instr_name' (instr_name.instr): running on 4 nodes (master is 'CI0021617', MPI version 3.1).
Detector: PSD_I=0.114429 PSD_ERR=0.000144614 PSD_N=626113 "PSD.dat"
Thanks for using McStasScript!
Thanks for using McStasScript!
Thanks for using McStasScript!
Thanks for using McStasScript!
loading system configuration



In [22]:
print(data)

[
McStasData: PSD type: 2D  I:0.114429 E:0.000144614 N:626113]


## Visualizing the instrument
It is possible to visualize the instrument using the visualization features in McStas / McXtrace. If done in a Jupyter Notebook, the visualization can be viewed in the notebook. The *show_instrument* method is used, and one can choose the format with the keyword *format* which can be either "window" or "webgl". When using window format a new window is opened with 2D visualization, and with the webgl format a 3D viewer is opened in a new browser tab. Be aware that the parameters set for the instrument is used for the visualization.

In [23]:
instrument.show_instrument()


loading system configuration

INFO: Using directory: "instr_name_mcdisplay"
INFO: Regenerating c-file: instr_name.c
CFLAGS=
INFO: Recompiling: ./instr_name.out
mccode-r.c:2837:3: warning: expression result unused [-Wunused-value]
*t0;
^~~
1 warning generated.
INFO: ===
instrument definition parsed
reading particle data...

Component                         source AT (0,0,0)    0 m from origin
Component                            PSD AT (0,0,1.9)    1.9 m from origin
Component                    source_slit AT (0,0,2)    2 m from origin
Component                          guide AT (0,0,2.1)    2.1 m from origin
Component                        guide_2 AT (0,0,12.11)    12.11 m from origin
Detector: PSD_I=0.116358 PSD_ERR=0.00841937 PSD_N=191 "PSD.dat"
Thanks for using McStasScript!
loading system configuration

starting particle parsing
ended particle parsing



In [24]:
instrument.show_instrument(format="window")

/Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1/bin/../tools/Python/mcdisplay/pyqtgraph/mcdisplay.py:292: DeprecationWarning: GraphicsWindow is deprecated, use GraphicsLayoutWidget instead,will be removed in 0.13
  window = pg.GraphicsWindow()

loading system configuration

INFO: Using directory: "instr_name_mcdisplay_0"
INFO: Regenerating c-file: instr_name.c
CFLAGS=
INFO: Recompiling: ./instr_name.out
mccode-r.c:2837:3: warning: expression result unused [-Wunused-value]
*t0;
^~~
1 warning generated.
INFO: ===
instrument definition parsed
reading particle data...

Component                         source AT (0,0,0)    0 m from origin
Component                            PSD AT (0,0,1.9)    1.9 m from origin
Component                    source_slit AT (0,0,2)    2 m from origin
Component                          guide AT (0,0,2.1)    2.1 m from origin
Component                        guide_2 AT (0,0,12.11)    12.11 m from origin
Detector: PSD_I=0.127933 PSD_ERR=0.0152909

When using the 3D view, use these controls to manipulate the view:

| Action | Effect on view |
| --- | --- |
| Hold left click and drag | Rotate |
| Hold right click and drag | Move |
| Hold mouse wheel and drag up/down | Zoom in/out |


## Dump and load an instrument object
It is possible to save an instrument object to disk and load it later. For now the name is still a required parameter, but it is overwritten by the loading process.

In [25]:
instrument.dump("dump_file_name.dmp")

'dump_file_name.dmp'

In [26]:
loaded_instrument = instr.McStas_instr("", dumpfile="dump_file_name.dmp")

In [27]:
loaded_instrument.print_components()
loaded_instrument.show_settings()

source      Source_div  AT      (0, 0, 0)     ABSOLUTE            
PSD         PSD_monitor AT      (0, 0, 1.9)   RELATIVE source     
source_slit Slit        AT      (0, 0, 2)     RELATIVE source     
guide       Guide       AT      (0, 0, 0.1)   RELATIVE source_slit
guide_2     Guide       AT      (0, 0, 10.01) RELATIVE guide       
                        ROTATED (0, 0.5, 0)   RELATIVE guide
Instrument settings:
  output_path:      instr_name_data
  run_path:         .
  package_path:     /Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1
  executable_path:  /Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1/bin/
  executable:       mcrun
  force_compile:    True
